In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [8]:
sample_query_cost = 1193
sample_query_timing = 185

In [9]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [10]:
%%sql result_1a <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [11]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [12]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [13]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [14]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.096..3.175 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.039..0.827 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.007..0.007 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.859 ms
 Execution Time: 3.260 ms
(8 rows)



In [15]:
inducted_hof_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.056..2.663 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.022..0.718 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.005..0.005 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.747 ms
 Execution Time: 2.742 ms
(8 rows)



In [16]:
provided_query_cost = 959
provided_query_timing = 12.589
your_query_cost = 959
your_query_timing = 11.613

In [17]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [18]:
q1b_part2 = ['A', 'B']

In [19]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [20]:
%%sql DROP VIEW IF EXISTS ca_schools;
CREATE VIEW ca_schools AS
SELECT * FROM schools WHERE schoolstate = 'CA'; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [21]:
%%sql DROP VIEW IF EXISTS ca_players;
CREATE VIEW ca_players AS
SELECT playerid, collegeplaying.schoolid FROM collegeplaying, ca_schools WHERE collegeplaying.schoolid = ca_schools.schoolid; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [22]:
%sql SELECT * from ca_players LIMIT 20; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
20 rows affected.


playerid,schoolid
abbotky01,ucsd
abbotky01,ucsd
abbotky01,longbeach
adamsbo04,ucla
adamsbo04,ucla
adamsbo04,ucla
adamsjo02,cacerri
adamsmi01,usc
adamswi02,stanford
adamswi02,stanford


In [23]:
%%sql DROP VIEW IF EXISTS x_players;
CREATE VIEW x_players AS
SELECT namefirst, namelast, people.playerid, schoolid FROM people, ca_players WHERE people.playerid = ca_players.playerid;  

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [24]:
%sql SELECT * from halloffame LIMIT 3; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
3 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None


In [25]:
%%sql DROP VIEW IF EXISTS h_players;
CREATE VIEW h_players AS
SELECT playerid, yearid FROM halloffame WHERE inducted = 'Y';   

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [26]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
SELECT namefirst, namelast, x_players.playerid, schoolid, h_players.yearid FROM x_players, h_players
WHERE x_players.playerid = h_players.playerid
ORDER BY yearid, playerid; 

SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [27]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [28]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [29]:
%%sql with_view <<
SELECT COUNT(playerid), schoolid, yearid FROM inducted_hof_ca 
GROUP by schoolid, yearid
ORDER BY yearid ASC;



 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [30]:
%%sql without_view <<
SELECT COUNT(x_players.playerid), schoolid, h_players.yearid FROM x_players, h_players
WHERE x_players.playerid = h_players.playerid
GROUP BY schoolid, h_players.yearid
ORDER BY yearid; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [31]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [32]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [33]:
with_view_query_str = "SELECT COUNT(playerid), schoolid, yearid FROM inducted_hof_ca GROUP by schoolid, yearid ORDER BY yearid ASC;"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                                   QUERY PLAN                                                                                    
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.73..325.75 rows=1 width=60) (actual time=5.636..5.648 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=325.73..325.74 rows=1 width=90) (actual time=5.629..5.632 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=325.71..325.72 rows=1 width=90) (actual time=5.561..5.568 rows=23 loops=1)
               ->  Sort  (cost=325.71..325.71 rows=1 width=326) (actual time=5.558..5.561 rows=23 loops=1)
                     Sort Key: halloffam

In [34]:
without_view_query_str = "SELECT COUNT(x_players.playerid), schoolid, h_players.yearid FROM x_players, h_players WHERE x_players.playerid = h_players.playerid GROUP BY schoolid, h_players.yearid ORDER BY yearid; "
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.71..325.73 rows=1 width=60) (actual time=4.743..4.754 rows=13 loops=1)
   Group Key: halloffame.yearid, collegeplaying.schoolid
   ->  Sort  (cost=325.71..325.71 rows=1 width=90) (actual time=4.735..4.738 rows=23 loops=1)
         Sort Key: halloffame.yearid, collegeplaying.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=0.85..325.70 rows=1 width=90) (actual time=1.777..4.679 rows=23 loops=1)
               ->  Nested Loop  (cost=0.57..319.95 rows=19 width=90) (actual time=0.155..4.210 rows=137 loops=1)
                     ->  Nested Loop  (cost=0.29..262.79 rows=21 width=80) (actual time=0.064

In [35]:
with_view_cost = 518
with_view_timing = 14.8
without_view_cost = 518
without_view_timing = 6.783

In [36]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [37]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS

SELECT namefirst, namelast, x_players.playerid, schoolid, h_players.yearid FROM x_players, h_players
WHERE x_players.playerid = h_players.playerid
ORDER BY yearid, playerid;

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [38]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [39]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [40]:
%%sql with_materialized_view <<

SELECT COUNT(playerid), schoolid, yearid FROM inducted_hof_ca _mat
GROUP by schoolid, yearid
ORDER BY yearid ASC;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [41]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [42]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [43]:
with_materialized_view_query_str = "SELECT COUNT(playerid), schoolid, yearid FROM inducted_hof_ca _mat GROUP by schoolid, yearid ORDER BY yearid ASC;"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                                                   QUERY PLAN                                                                                    
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.73..325.75 rows=1 width=60) (actual time=4.878..4.894 rows=13 loops=1)
   Group Key: _mat.yearid, _mat.schoolid
   ->  Sort  (cost=325.73..325.74 rows=1 width=90) (actual time=4.869..4.872 rows=23 loops=1)
         Sort Key: _mat.yearid, _mat.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on _mat  (cost=325.71..325.72 rows=1 width=90) (actual time=4.797..4.806 rows=23 loops=1)
               ->  Sort  (cost=325.71..325.71 rows=1 width=326) (actual time=4.794..4.797 rows=23 loops=1)
                     Sort Key: halloffame.yearid, people.playerid
                     Sort Met

In [44]:
with_materialized_view_cost = 519
with_materialized_view_timing = 6.077

In [45]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [46]:
q1e_part4 = ['D','J']

In [47]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

Once a user creates a materialized view, it is saved on the disk for future reference, however, for a simple view every time the user referes back to the view - it is regenerated causing it to take more time and increasing the cost when it is called again. A, B, C are invalid options because the cost for with view and without view is approx the same ~ 325 and each time we call either operations, all tasks are performed in the background. E, F, G these options show an inverse relation between cost and time, hence I did not select them. H and I show that aren't correct as Materialized views drastically reduce the time. 

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [48]:
query_view_str = "SELECT * FROM inducted_hof_ca;"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                          QUERY PLAN                                                                           
---------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=325.71..325.71 rows=1 width=326) (actual time=5.299..5.302 rows=23 loops=1)
   Sort Key: halloffame.yearid, people.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=0.85..325.70 rows=1 width=326) (actual time=1.743..5.232 rows=23 loops=1)
         ->  Nested Loop  (cost=0.57..319.95 rows=19 width=326) (actual time=0.152..4.704 rows=137 loops=1)
               ->  Nested Loop  (cost=0.29..262.79 rows=21 width=316) (actual time=0.052..2.985 rows=323 loops=1)
                     ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.024..0.768 rows=323 loops=1)
                           Filter: ((inducted)::text

In [49]:
query_view_cost = 527
query_view_timing = 5.892

In [50]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [51]:
%%sql result_2b <<
SELECT * FROM inducted_hof_ca WHERE yearid > '2010';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [52]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > '2010';"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                          QUERY PLAN                                                                          
--------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=185.88..185.89 rows=1 width=326) (actual time=1.376..1.379 rows=8 loops=1)
   Sort Key: halloffame.yearid, people.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..185.87 rows=1 width=326) (actual time=0.659..1.329 rows=8 loops=1)
         ->  Nested Loop  (cost=0.57..184.05 rows=6 width=326) (actual time=0.637..1.198 rows=27 loops=1)
               ->  Nested Loop  (cost=0.29..165.00 rows=7 width=316) (actual time=0.539..0.924 rows=31 loops=1)
                     ->  Seq Scan on halloffame  (cost=0.00..106.86 rows=7 width=42) (actual time=0.506..0.554 rows=31 loops=1)
                           Filter: ((yearid > 2010) AND ((ind

In [53]:
query_view_with_filter_cost = 209
query_view_with_filter_timing = 1.25

In [54]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [55]:
q2c = ['A', 'E']

In [56]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

A, E: 
Applying a filter drastically reduces the time and cost as the amount of data that is being analyzed to be 'joined' is reduced. 
Other Options: Based on the cost and time, we can clearly see there is a difference due to the filter and hence ruling out the other options. 

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [57]:
%%sql q3a_join <<
SELECT * FROM people 
INNER JOIN collegeplaying
ON people.playerid = collegeplaying.playerid; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [58]:
q3a_query_str = "SELECT * FROM people INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                                         QUERY PLAN                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.120..55.753 rows=17350 loops=1)
   Merge Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Index Scan using master_pkey on people  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.039..7.252 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying  (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.074..29.265 rows=17350 loops=1)
         Heap Fetches: 17350
 Planning Time: 1.046 ms
 Execution Time: 56.833 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [59]:
q3a = ['C']

In [60]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [61]:
%%sql q3b_join <<
SELECT * FROM people 
INNER JOIN collegeplaying
ON people.playerid = collegeplaying.playerid
ORDER BY people.playerid; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [62]:
q3b_query_str = "SELECT * FROM people INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid ORDER BY people.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                        QUERY PLAN                                                                        
----------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..3184.67 rows=17350 width=1719) (actual time=0.078..18.855 rows=17350 loops=1)
   Merge Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Index Scan using master_pkey on people  (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.028..4.488 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.045..2.416 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 0.845 ms
 Execution Time: 19.614 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [63]:
q3b = ["B"]

In [64]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [65]:
%%sql q3c_join <<
SELECT * 
FROM people as p 
INNER JOIN people ON p.playerid != people.playerid
LIMIT 1000; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [66]:
q3c_query_str = "SELECT * FROM people as p INNER JOIN people ON p.playerid != people.playerid LIMIT 1000; "
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                         QUERY PLAN                                                          
-----------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.034..1.025 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.033..0.955 rows=1000 loops=1)
         Join Filter: ((p.playerid)::text <> (people.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.017..0.018 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.008..0.506 rows=1001 loops=1)
               ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.001..0.107 rows=1001 loops=1)
 Planning Time: 0.871 ms
 Execution Time: 1.199 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [67]:
q3c = ['A']

In [68]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

3a: A hash join was used because that resulted in equi join and the answer did not require any sorting

3b: A merge join is used because the resulting table is required in a sorted manner. 

3c: As we are LIMITING the result to 1000 rows and we are just targeting a smaller subset of rows, the ideal join is Nested Loop join. 

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [69]:
%%sql DROP VIEW IF EXISTS tb1;
CREATE VIEW tb1 AS
SELECT * FROM appearances
NATURAL JOIN salaries; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [70]:
%%sql result_4a <<
SELECT playerid, AVG(salary) FROM tb1 WHERE g_batting = '10'
GROUP BY playerid; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [71]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [72]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [73]:
result_4a_query_str = "SELECT playerid, AVG(salary) FROM tb1 WHERE g_batting = '10' GROUP BY playerid;"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3637.50..3637.53 rows=1 width=17) (actual time=24.453..24.526 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=3637.50..3637.51 rows=1 width=17) (actual time=24.441..24.452 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2900.72..3637.49 rows=1 width=17) (actual time=15.754..24.269 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salar

In [74]:
result_4a_cost = 3801
result_4a_timing = 68.502

In [75]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [76]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [77]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2386.96..2386.98 rows=1 width=17) (actual time=11.348..11.419 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=2386.96..2386.96 rows=1 width=17) (actual time=11.338..11.348 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1650.17..2386.95 rows=1 width=17) (actual time=2.973..11.166 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salari

In [78]:
result_4b_cost = 2370
result_4b_timing = 12.825

In [79]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [80]:
%%sql 
DROP INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [81]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries (salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [82]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3637.50..3637.53 rows=1 width=17) (actual time=21.783..21.860 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=3637.50..3637.51 rows=1 width=17) (actual time=21.771..21.782 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2900.72..3637.49 rows=1 width=17) (actual time=13.266..21.569 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salar

In [83]:
result_4c_cost = 3638
result_4c_timing = 26.596

In [84]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [85]:
q4d_part1 = ['D','E' ]

In [86]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

D, E: 
As the index is created for g_batting and the where clause is applied for g_batting - that will result in the faster and more efficient results. 

Other Options: The salary index does not matter as the where clause does not account for salary, hence the salary index does not make sense. 

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [87]:
%%sql 
DROP INDEX salary_idx; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [88]:
%%sql result_4ei <<
SELECT playerid,yearid, salary FROM tb1 WHERE g_batting = '10' AND g_all = '10';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [89]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [90]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [91]:
result_4ei_query_str = "SELECT playerid,yearid, salary FROM tb1 WHERE g_batting = '10' AND g_all = '10';"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3308.72 rows=1 width=21) (actual time=9.503..16.976 rows=120 loops=1)
   ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=22 width=20) (actual time=0.015..13.545 rows=1289 loops=1)
         Filter: ((g_batting = 10) AND (g_all = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries  (cost=0.29..7.95 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((yearid = appearances.yearid) AND ((teamid)::text = (appearances.teamid)::text) AND ((lgid)::text = (appearances.lgid)::text) AND ((pla

In [92]:
result_4ei_cost = 3300
result_4ei_timing = 16.306

In [93]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [94]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [95]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.91..1800.63 rows=1 width=21) (actual time=3.011..5.854 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=18.62..1625.75 rows=22 width=20) (actual time=0.313..2.586 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..18.61 rows=1376 width=0) (actual time=0.189..0.189 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries  (cost=0.

In [96]:
result_4eii_with_index_cost = 1777
result_4eii_with_index_timing = 27.083

In [97]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [98]:
%%sql 
DROP INDEX g_batting_idx; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [99]:
%%sql result_4eiii <<
SELECT playerid,yearid, salary FROM tb1 WHERE g_batting = '10' OR g_all = '10';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [100]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [101]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [102]:
result_4eiii_query_str = "SELECT playerid,yearid, salary FROM tb1 WHERE g_batting = '10' OR g_all = '10';"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3193.88..3930.65 rows=1 width=21) (actual time=16.525..26.278 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.013..2.457 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=3002 width=20) (actual time=16.167..16.170 rows=

In [103]:
result_4eiii_cost = 3929
result_4eiii_timing = 53.263

In [104]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [105]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx
ON appearances (g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [106]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3193.88..3930.65 rows=1 width=21) (actual time=15.139..23.911 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.015..2.217 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=3002 width=20) (actual time=14.852..14.855 rows=

In [107]:
result_4eiv_with_index_cost = 3192
result_4eiv_with_index_timing = 27.155

In [108]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [109]:
%%sql 
DROP INDEX g_batting_index; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.UndefinedObject) index "g_batting_index" does not exist

[SQL: DROP INDEX g_batting_index;]
(Background on this error at: https://sqlalche.me/e/14/f405)


#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [110]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances (g_batting, g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [111]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2871.61..3608.38 rows=1 width=21) (actual time=4.237..12.472 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.015..2.091 rows=26428 loops=1)
   ->  Hash  (cost=2811.57..2811.57 rows=3002 width=20) (actual time=4.027..4.031 rows=165

In [112]:
result_4ev_multiple_col_index_cost = 2875
result_4ev_multiple_col_index_timing = 13.470

In [113]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [114]:
q4e_part6 = ['C','I']

In [115]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

A,B: I did choose these options as the time and cost seem to be correlated. 
C: Adding an index makes the 

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [116]:
%%sql result_5a_min << 
SELECT MIN(salary) FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [117]:
result_5a_min_query_str = "SELECT MIN(salary) FROM salaries;"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.459..4.460 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.011..2.008 rows=26428 loops=1)
 Planning Time: 0.537 ms
 Execution Time: 4.537 ms
(4 rows)



In [118]:
result_5a_min_query_cost = 525
result_5a_min_query_timing = 6.969

In [119]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [120]:
%%sql result_5a_avg <<
SELECT AVG(salary) FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [121]:
result_5a_avg_query_str = "SELECT AVG(salary) FROM salaries;"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.818..5.820 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.012..2.318 rows=26428 loops=1)
 Planning Time: 0.351 ms
 Execution Time: 5.901 ms
(4 rows)



In [122]:
result_5a_avg_query_cost = 525
result_5a_avg_query_timing = 5.179

In [123]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [124]:
%%sql 
CREATE INDEX salary_idx
ON salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [125]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.062..0.063 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.059..0.060 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.058..0.058 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.635 ms
 Execution Time: 0.113 ms
(8 rows)



In [126]:
result_5b_min_query_cost = 0.32
result_5b_min_query_timing = 0.123

In [127]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [128]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=8.093..8.095 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..3.188 rows=26428 loops=1)
 Planning Time: 0.655 ms
 Execution Time: 8.208 ms
(4 rows)



In [129]:
result_5b_avg_query_cost = 525.35
result_5b_avg_query_timing = 4.973

In [130]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [131]:
%%sql 
DROP INDEX salary_idx; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [132]:
q5c = ['D']

In [133]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

D: Indexing through the columns to find min and max allows us to directly index into the required metrics and does not require the system to go over all the rows. 

Other options: To get AVG or COUNT, the system is required to go over all the rows irrespective of index, hence adding an index does not change the time. 

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [134]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > '500'; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [135]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [136]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [137]:
result_6a_query_str = "SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > '500'; "
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=8726 width=21) (actual time=0.278..13.793 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.389 ms
 Execution Time: 14.162 ms
(5 rows)



In [138]:
result_6a_cost = 2884
result_6a_timing = 28.180

In [139]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [140]:
%%sql 
CLUSTER batting USING batting_pkey; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [141]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=8726 width=21) (actual time=0.021..14.505 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.475 ms
 Execution Time: 14.896 ms
(5 rows)



In [142]:
result_6b_cost = 2878
result_6b_timing = 6.3

In [143]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [144]:
%%sql 
CREATE INDEX ab_idx
ON batting (ab);

CLUSTER batting USING ab_idx; 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [145]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                      
----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=99.92..1783.99 rows=8726 width=21) (actual time=0.314..1.996 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=135
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..97.74 rows=8726 width=0) (actual time=0.283..0.284 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.600 ms
 Execution Time: 2.406 ms
(7 rows)



In [146]:
result_6c_cost = 1786
result_6c_timing = 2.421

In [147]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [148]:
q6d = ['A','D']

In [149]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

As the filter was applyed on the colum 'ab', applying an index on ab makes the system more faster and efficient, however as batting_pkey isn't used for the filter applying an index on that does not make any differnce to the result. 

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [150]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [151]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 4.23 ms, sys: 2.11 ms, total: 6.34 ms
Wall time: 3.61 s


[]

In [152]:
result_7a_timing = 3.79

In [153]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [154]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [155]:
%%sql 
CREATE INDEX salary_idx
ON salaries (salary); 

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [156]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 5.87 ms, sys: 1.72 ms, total: 7.6 ms
Wall time: 6.46 s


[]

In [157]:
result_7b_timing = 5.59

In [158]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

Each time we add an index, the system takes up time to sort the rows accordingly - however, if an index is added unnecessarily the time taken actually increases because of redundant processing hence slowing down the system. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

1. Indexing: The above examples showed how indexes can drastically reduce the time taken and make the process more efficient by sorting the rows and allowing the system to target the required data. For instance, the MIN and MAX functions are faster with indexing. However, indexing can also be redundant if the functions are not relevant and can increase the time taken. 

2. Using Filters: Using filters can help specify the information required and reduce time before the join operations.

3. Clustering: Allows us to index to groups and to minimize the time taken to return data. 

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [159]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [160]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1bi results: All test cases passed!

q1bii results: All test cases passed!

q1c results: All test cases passed!

q1di results: All test cases passed!

q1dii results: All test cases passed!

q1ei results: All test cases passed!

q1eii results: All test cases passed!

q1eiii results: All test cases passed!

q1eiv results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q4ai results: All test cases passed!

q4aii results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q4di results: All test cases passed!

q4ei_part1 results: All test cases passed!

q4ei_part2 results: All test cases passed!

q4eii results: All test cases passed!

q4eiii_part1 results: All test cases passe

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [161]:
# Save your notebook first, then run this cell to export your submission.
grader.export()